In [1]:
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
import time
from gseapy.plot import barplot, dotplot

wk_dir = '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-3/data/3-15_DMSO_TGF_BvsTshnc/'

### need to set species here

In [2]:
# ‘Human’, ‘Mouse’, ‘Yeast’, ‘Fly’, ‘Fish’, ‘Worm’
speices = 'Human'

* add file path
* add df.name
* append to list
* **NOTE** need to pay attention to gene name in tsv and up down case issue
* Seems enricr has a list **size limit** and will casuse issue

In [3]:
# add file path
# add df.name
# append to list
DEG_DMSO_nc_vs_kd = '/Users/jplab/Desktop/snakepipes_Data/results_mRNA_seq_FOXA1_KD_NC_2022221/DESeq2_samplesheet_FOXA1-DMSO-NCvsKD/DEseq_basic_DEresults_counts_DESeq2.normalized_noVersion.tsv'
DEG_TGF_nc_vs_kd = '/Users/jplab/Desktop/snakepipes_Data/results_mRNA_seq_FOXA1_KD_NC_2022221/DESeq2_samplesheet_FOXA1-TGFb-NCvsKD/DEseq_basic_DEresults_counts_DESeq2.normalized_noVersion.tsv'
DEG_shFOXA1_B14_vs_TGF = '/Users/jplab/Desktop/snakepipes_Data/results_mRNA_seq_FOXA1_KD_NC_2022221/DESeq2_samplesheet_FOXA1KD-B14vsTGFb/DEseq_basic_DEresults_counts_DESeq2.normalized_noVersion.tsv'
DEG_shNC_B14_vs_TGF = '/Users/jplab/Desktop/snakepipes_Data/results_mRNA_seq_FOXA1_KD_NC_2022221/DESeq2_samplesheet_shNC-B14vsTGFb/DEseq_basic_DEresults_counts_DESeq2.normalized_noVersion.tsv'


df_DMSO_nc_vs_kd = pd.read_csv(DEG_DMSO_nc_vs_kd,header=0, sep="\t",index_col=False)
df_DMSO_nc_vs_kd.name = 'df_DMSO_nc_vs_kd'

df_TGF_nc_vs_kd = pd.read_csv(DEG_TGF_nc_vs_kd,header=0, sep="\t",index_col=False)
df_TGF_nc_vs_kd.name = 'df_TGF_nc_vs_kd'

df_shFOXA1_B14_vs_TGF = pd.read_csv(DEG_shFOXA1_B14_vs_TGF,header=0, sep="\t",index_col=False)
df_shFOXA1_B14_vs_TGF.name = 'df_shFOXA1_B14_vs_TGF'

df_shNC_B14_vs_TGF = pd.read_csv(DEG_shNC_B14_vs_TGF,header=0, sep="\t",index_col=False)
df_shNC_B14_vs_TGF.name = 'df_shNC_B14_vs_TGF'

df_list = []
df_list.append(df_DMSO_nc_vs_kd)
df_list.append(df_TGF_nc_vs_kd)
df_list.append(df_shFOXA1_B14_vs_TGF)
df_list.append(df_shNC_B14_vs_TGF)

# first use dict to save loop filter results
dict = {}
for df in df_list:
    print(df.name)
    df_name = df.name

    dict[df_name + '_UP_genes'] = df.query("Status == 'UP'")
    dict[df_name + '_DOWN_genes'] = df.query("Status == 'DOWN'")
    # dict[df_name + '_UP_and_DOWN_genes'] = df.query("change == 'Down' or change == 'Up'")
print(dict.keys())

# then loop dict
dict_gene_list = {}
for df_name, df in dict.items():
    # sort by fold change
    df = df.sort_values(by=['log2FoldChange'], ascending=False)

    # drop dup in symbol column
    df_ori_row_num = len(df.index)
    df = df.drop_duplicates(subset=['external_gene_name'], keep='first')
    df_dupDrop_row_num = len(df.index)
    print(df_name + ': ' + str(df_ori_row_num - df_dupDrop_row_num) + ' rows of dup are dropped')

    # must drop na, or will throw error in enrichr method
    df = df['external_gene_name'].dropna()
    df_dropNa_row_num = len(df.index)
    print(df_name + ': ' + str(df_dupDrop_row_num - df_dropNa_row_num) + ' rows of NA are dropped')

    # convert to list
    dict_gene_list[df_name.split('df_')[1]] = df.tolist()



df_DMSO_nc_vs_kd
df_TGF_nc_vs_kd
df_shFOXA1_B14_vs_TGF
df_shNC_B14_vs_TGF
dict_keys(['df_DMSO_nc_vs_kd_UP_genes', 'df_DMSO_nc_vs_kd_DOWN_genes', 'df_TGF_nc_vs_kd_UP_genes', 'df_TGF_nc_vs_kd_DOWN_genes', 'df_shFOXA1_B14_vs_TGF_UP_genes', 'df_shFOXA1_B14_vs_TGF_DOWN_genes', 'df_shNC_B14_vs_TGF_UP_genes', 'df_shNC_B14_vs_TGF_DOWN_genes'])
df_DMSO_nc_vs_kd_UP_genes: 0 rows of dup are dropped
df_DMSO_nc_vs_kd_UP_genes: 0 rows of NA are dropped
df_DMSO_nc_vs_kd_DOWN_genes: 0 rows of dup are dropped
df_DMSO_nc_vs_kd_DOWN_genes: 0 rows of NA are dropped
df_TGF_nc_vs_kd_UP_genes: 0 rows of dup are dropped
df_TGF_nc_vs_kd_UP_genes: 0 rows of NA are dropped
df_TGF_nc_vs_kd_DOWN_genes: 0 rows of dup are dropped
df_TGF_nc_vs_kd_DOWN_genes: 0 rows of NA are dropped
df_shFOXA1_B14_vs_TGF_UP_genes: 0 rows of dup are dropped
df_shFOXA1_B14_vs_TGF_UP_genes: 0 rows of NA are dropped
df_shFOXA1_B14_vs_TGF_DOWN_genes: 1 rows of dup are dropped
df_shFOXA1_B14_vs_TGF_DOWN_genes: 0 rows of NA are dropped
df_s

In [4]:
# get enrichr lib name from GSEApy
lib_names = gp.get_library_name()
lib_names
print(lib_names)

['ARCHS4_Cell-lines', 'ARCHS4_IDG_Coexp', 'ARCHS4_Kinases_Coexp', 'ARCHS4_TFs_Coexp', 'ARCHS4_Tissues', 'Achilles_fitness_decrease', 'Achilles_fitness_increase', 'Aging_Perturbations_from_GEO_down', 'Aging_Perturbations_from_GEO_up', 'Allen_Brain_Atlas_10x_scRNA_2021', 'Allen_Brain_Atlas_down', 'Allen_Brain_Atlas_up', 'Azimuth_Cell_Types_2021', 'BioCarta_2013', 'BioCarta_2015', 'BioCarta_2016', 'BioPlanet_2019', 'BioPlex_2017', 'CCLE_Proteomics_2020', 'CORUM', 'COVID-19_Related_Gene_Sets', 'COVID-19_Related_Gene_Sets_2021', 'Cancer_Cell_Line_Encyclopedia', 'CellMarker_Augmented_2021', 'ChEA_2013', 'ChEA_2015', 'ChEA_2016', 'Chromosome_Location', 'Chromosome_Location_hg19', 'ClinVar_2019', 'DSigDB', 'Data_Acquisition_Method_Most_Popular_Genes', 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019', 'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019', 'Descartes_Cell_Types_and_Tissue_2021', 'DisGeNET', 'Disease_Perturbations_from_GEO_down', 'Disease_Perturbations_from_GEO_up', 'Disease_Signatures

In [5]:
# set target libs to be used in GSEApy
target_libs = [
    'WikiPathway_2021_Human',
    'KEGG_2021_Human',
    'MSigDB_Hallmark_2020',

    'GO_Biological_Process_2021',
    'GO_Cellular_Component_2021',
    'GO_Molecular_Function_2021',
    
    'GTEx_Tissue_Expression_Down',
    'GTEx_Tissue_Expression_Up',
    'Kinase_Perturbations_from_GEO_down',
    'Kinase_Perturbations_from_GEO_up',
    'Epigenomics_Roadmap_HM_ChIP-seq',
    'Enrichr_Submissions_TF-Gene_Coocurrence',
    'CCLE_Proteomics_2020',
    'MSigDB_Oncogenic_Signatures',
    'Old_CMAP_down',
    'Old_CMAP_up',
    'Phosphatase_Substrates_from_DEPOD',
    'PPI_Hub_Proteins',
    'RNA-Seq_Disease_Gene_and_Drug_Signatures_from_GEO',
    'RNAseq_Automatic_GEO_Signatures_Human_Down',
    'RNAseq_Automatic_GEO_Signatures_Human_Up',
    'TRANSFAC_and_JASPAR_PWMs',
    'Tissue_Protein_Expression_from_Human_Proteome_Map',
    ]

target_libs = [
    'MSigDB_Hallmark_2020',

    'GO_Biological_Process_2021',
    'GO_Cellular_Component_2021',
    'GO_Molecular_Function_2021',

    'Mouse_Gene_Atlas',
    'HDSigDB_Mouse_2021',
    'WikiPathways_2019_Mouse',
    'KEGG_2019_Mouse',
    'RNAseq_Automatic_GEO_Signatures_Mouse_Down',
    'RNAseq_Automatic_GEO_Signatures_Mouse_Up',

    'Kinase_Perturbations_from_GEO_down',
    'Kinase_Perturbations_from_GEO_up',
    'Enrichr_Submissions_TF-Gene_Coocurrence',
    'CCLE_Proteomics_2020',
    'MSigDB_Oncogenic_Signatures',
    'Old_CMAP_down',
    'Old_CMAP_up',
    'Phosphatase_Substrates_from_DEPOD',
    'PPI_Hub_Proteins',
    'TRANSFAC_and_JASPAR_PWMs'
    ]      

# say something
# careful here, if i use test as description, it works; if i use 'nc_B14_vs_TGF and kd_B14_vs_TGF' as description, it will throw error on gp.enrichr method
# description = 'nc_B14_vs_TGF and kd_B14_vs_TGF'
description = 'test'

In [6]:
for lib_name in target_libs:
    print(lib_name)
    
    for gene_list_name, gene_list in dict_gene_list.items():
        print(gene_list_name)
        # print(gene_list)
        try :
            enr = gp.enrichr(
                        gene_list=gene_list,
                        gene_sets=lib_name,
                        organism=speices,
                        description=description,
                        outdir=wk_dir+'barplots_and_tabs/'+gene_list_name,
                        # no_plot=False,
                        cutoff=0.05,
                        verbose=True
                        )
            # print(enr.res2d)   
            # time.sleep(10)
        except Exception as e:
            print('error:', e)

        ## to save your figure, make sure that ``ofname`` is not None
        # title = ' and '.join(target_libs) + ' ' + gene_list_name
        title = lib_name + ' ' + gene_list_name
        print(title)

        try :
            print('i got res2d: {0}'.format(enr.res2d))
            dotplot(enr.res2d,title=title,
                top_term=20,
                cmap='viridis_r',
                ofname=wk_dir + title+'.png',
                )
            # time.sleep(10)                
        except Exception as e2 :
            print('error2: ',e2)

WikiPathway_2021_Human
DMSO_nc_vs_kd_UP_genes


2022-03-15 13:46:12,817 Connecting to Enrichr Server to get latest library names
2022-03-15 13:46:12,818 Analysis name: test, Enrichr Library: WikiPathway_2021_Human
2022-03-15 13:46:19,796 Save file of enrichment results: Job Id:4ce0c892454764280a232488ac561e49
2022-03-15 13:46:20,510 Done.



WikiPathway_2021_Human DMSO_nc_vs_kd_UP_genes
i got res2d:                    Gene_set  \
0    WikiPathway_2021_Human   
1    WikiPathway_2021_Human   
2    WikiPathway_2021_Human   
3    WikiPathway_2021_Human   
4    WikiPathway_2021_Human   
..                      ...   
501  WikiPathway_2021_Human   
502  WikiPathway_2021_Human   
503  WikiPathway_2021_Human   
504  WikiPathway_2021_Human   
505  WikiPathway_2021_Human   

                                                  Term Overlap       P-value  \
0    Cholesterol metabolism (includes both Bloch an...   22/46  1.438988e-15   
1                Nuclear Receptors Meta-Pathway WP2882  52/319  2.074331e-11   
2               Cholesterol Biosynthesis Pathway WP197   11/15  3.073607e-11   
3                                  NRF2 pathway WP2884  31/146  3.420795e-10   
4              p53 transcriptional gene network WP4963   19/67  5.901642e-09   
..                                                 ...     ...           ...   
501     

2022-03-15 13:46:22,898 Connecting to Enrichr Server to get latest library names
2022-03-15 13:46:22,899 Analysis name: test, Enrichr Library: WikiPathway_2021_Human
2022-03-15 13:46:31,318 Save file of enrichment results: Job Id:1c866d6eea4276b38e8f60eabf0e9bcd
2022-03-15 13:46:31,462 Done.



WikiPathway_2021_Human DMSO_nc_vs_kd_DOWN_genes
i got res2d:                    Gene_set  \
0    WikiPathway_2021_Human   
1    WikiPathway_2021_Human   
2    WikiPathway_2021_Human   
3    WikiPathway_2021_Human   
4    WikiPathway_2021_Human   
..                      ...   
499  WikiPathway_2021_Human   
500  WikiPathway_2021_Human   
501  WikiPathway_2021_Human   
502  WikiPathway_2021_Human   
503  WikiPathway_2021_Human   

                                                  Term Overlap       P-value  \
0                 Retinoblastoma gene in cancer WP2446   45/87  5.152814e-31   
1                                mRNA Processing WP411  44/126  1.063983e-21   
2                                DNA Replication WP466   25/42  7.744558e-20   
3                                     Cell cycle WP179  38/120  2.838497e-17   
4                            DNA Mismatch Repair WP531   17/23  2.662550e-16   
..                                                 ...     ...           ...   
499   

...

2022-03-15 13:47:50,952 Connecting to Enrichr Server to get latest library names
2022-03-15 13:47:50,953 Analysis name: test, Enrichr Library: KEGG_2021_Human
2022-03-15 13:47:56,495 Save file of enrichment results: Job Id:2eb7e5aaebbcbbf933f612f4bc3600c8
2022-03-15 13:47:56,632 Done.



KEGG_2021_Human TGF_nc_vs_kd_UP_genes
i got res2d:             Gene_set                                     Term Overlap  \
0    KEGG_2021_Human                                 Lysosome  26/128   
1    KEGG_2021_Human                    p53 signaling pathway   16/73   
2    KEGG_2021_Human                 ECM-receptor interaction   17/88   
3    KEGG_2021_Human                  Proteoglycans in cancer  26/205   
4    KEGG_2021_Human                     Steroid biosynthesis    6/20   
..               ...                                      ...     ...   
280  KEGG_2021_Human  Neuroactive ligand-receptor interaction   7/341   
281  KEGG_2021_Human                Oxidative phosphorylation   1/133   
282  KEGG_2021_Human                            RNA transport   2/186   
283  KEGG_2021_Human                                 Ribosome   1/158   
284  KEGG_2021_Human                               Alcoholism   1/186   

          P-value  Adjusted P-value  Old P-value  Old Adjusted P-value  

2022-03-15 13:47:58,648 Connecting to Enrichr Server to get latest library names
2022-03-15 13:47:58,649 Analysis name: test, Enrichr Library: KEGG_2021_Human
2022-03-15 13:48:04,124 Save file of enrichment results: Job Id:9fb07eb23d04a557897781e735756eb0
2022-03-15 13:48:04,258 Done.



KEGG_2021_Human TGF_nc_vs_kd_DOWN_genes
i got res2d:             Gene_set                                     Term Overlap  \
0    KEGG_2021_Human                               Cell cycle  39/124   
1    KEGG_2021_Human                          DNA replication   21/36   
2    KEGG_2021_Human                          Mismatch repair   11/23   
3    KEGG_2021_Human                   Fanconi anemia pathway   14/54   
4    KEGG_2021_Human                     Base excision repair   11/33   
..               ...                                      ...     ...   
244  KEGG_2021_Human                        Alzheimer disease   5/369   
245  KEGG_2021_Human  Neuroactive ligand-receptor interaction   4/341   
246  KEGG_2021_Human  Neutrophil extracellular trap formation   1/189   
247  KEGG_2021_Human         Herpes simplex virus 1 infection   3/498   
248  KEGG_2021_Human                   Olfactory transduction   2/440   

          P-value  Adjusted P-value  Old P-value  Old Adjusted P-value

2022-03-15 13:48:06,228 Connecting to Enrichr Server to get latest library names
2022-03-15 13:48:06,230 Analysis name: test, Enrichr Library: KEGG_2021_Human


### Also could input target_libs whole list to gene_sets; eg gene_sets=target_libs, this will combine resluts to one

In [ ]:
for gene_list_name, gene_list in dict_gene_list.items():
    print(gene_list_name)

name = 'kd_B14_vs_TGFb_UP_genes'
list = dict_gene_list[name]
enr = gp.enrichr(
                gene_list=list,
                gene_sets=target_libs,
                organism='Human',
                description=description,
                outdir=wk_dir+'enrichr_plots/'+name,
                # no_plot=False,
                cutoff=0.05,
                verbose=True
                )
# print(enr.res2d)   


# to save your figure, make sure that ``ofname`` is not None
title = ' and '.join(target_libs) + ' ' + name
# print(title)

dotplot(enr.res2d,title=title,
        top_term=20,
        cmap='viridis_r',
        # ofname=wk_dir + title+'.png',
        )   